# A catalog of Torch modules
The objective of this notebook is to present building blocks of neural networks in conjunction with Torch modules that provide them.  

- we classify transfer (a.k.a. activation) functions into
  - intrinsically scalar: Sigmoid, SoftPlus, Tanh, HardTanh, SoftSign, ReLU
  - intrinsically vector: SoftMax, LogSoftMax
- these can all be used with the Linear module (which provides the input weights to the layer)
- we present two loss functions: mean square error and negative log likelihood

In [ ]:
-- let's discuss neural networks and layers in torch
require 'nn'
require 'utils'
Plot = require 'itorch.Plot'
print('ready')

In [ ]:
m = nn.Linear(10,4)
print(m)

In [ ]:
print(m.bias)

In [ ]:
itorch.image(image.scale(m.weight,200,'simple'))

In [ ]:
x = torch.randn(10)
y = m:forward(x)
print(y)

In [ ]:
print(m.weight * x + m.bias)

In [ ]:
-- we will use x as input to the layer [change this if you want to zoom in/out on some range]
xs = torch.range(-50,50) / 10

## Intrinsically scalar transfer functions
- [Overview](https://github.com/torch/nn/blob/master/doc/transfer.md)
- Tanh = hyperbolic tangent [y = (exp(x) - exp(-x)) / (exp(x) + exp(-x))]
- HardTanh - a piecewise linear approximation to Tanh
- Sigmoid layer [y = 1 / (1 + exp(-x))]
- ReLU layer - Rectified Linear layer [y = max(x, 0)], [ReLU/PReLU results](http://arxiv.org/pdf/1502.01852v1.pdf)
- SoftPlus layer - a 'soft' version of ReLU

In [ ]:
ys1 = nn.Tanh():forward(xs)
ys2 = nn.HardTanh():forward(xs)

ys3 = nn.SoftSign():forward(xs)
ys4 = nn.Sigmoid():forward(xs)

p = Plot():line(xs,ys1,'red','Tanh'):line(xs,ys2,'green','HardTanh')
p:line(xs,ys3,'blue','SoftSign'):line(xs,ys4,'black', 'Sigmoid')
p:title('Tanh/HardTanh/Sigmoid activation functions'):legend(true):draw()

In [ ]:
ys4 = nn.ReLU():forward(xs)
ys5 = nn.SoftPlus():forward(xs)
p = Plot():line(xs,ys4,'red','ReLU'):line(xs,ys5,'blue','SoftPlus'):title('ReLU and SoftPlus transfer functions'):legend(true):draw()

# Convolutional layers
- general info on [convolutional layers](https://github.com/torch/nn/blob/master/doc/convolution.md)
- [nn.TemporalConvolution](https://github.com/torch/nn/blob/master/doc/convolution.md#nn.TemporalConvolution) is a 1D convolutional operator suitable for e.g. time series
- [nn.SpatialConvolution](https://github.com/torch/nn/blob/master/doc/convolution.md#spatialconvolution) layer is a 2D convolution (for use with images)
- max pooling and subsampling layers

In [ ]:
conv = nn.TemporalConvolution(1, 1, 2)
print(conv)
conv.weight[1][1] = -1
conv.weight[1][2] = 1
conv.bias[1] = 0
print(conv.weight)
print(conv.bias)

In [ ]:
xs = torch.rand(50,1) * 0.2
xs[{{15,30},1}] = xs[{{15,30},1}] + 1
ys = conv:forward(xs)
noutput = (xs:size(1) - 2) / 1 + 1

In [ ]:
-- reshape the vectors for plotting
time = torch.range(1,noutput)
xsp = xs[{{2,noutput+1},{}}]:reshape(noutput)
ysp = ys:reshape(noutput)

In [ ]:
p = Plot():title('Effect of 1D conv [shifted]'):line(time,xsp+1,'blue','input'):line(time, ysp-1,'red','output')
p:xaxis('samples'):yaxis('signal'):legend(true):draw()

In [ ]:
-- snippet shamelessly ripped from https://github.com/torch/tutorials/blob/master/A_datasets/mnist.lua 
-- load the training data (downloads when first used)
tar = 'http://torch7.s3-website-us-east-1.amazonaws.com/data/mnist.t7.tgz'

if not paths.dirp('mnist.t7') then
   print('==> downloading dataset')
   os.execute('wget ' .. tar)
   os.execute('tar xvf ' .. paths.basename(tar))
end
test_file = 'mnist.t7/test_32x32.t7'

----------------------------------------------------------------------
print('==> loading dataset')

-- We load the dataset from disk, it's straightforward
test_data = torch.load(test_file,'ascii')

In [ ]:
-- Let's plot one of the examples
inp2d = test_data.data[{244}]:type('torch.DoubleTensor')
print('Size of the input is ' .. inp2d:size(1) .. ' x ' .. inp2d:size(2) .. ' x ' .. inp2d:size(3))
itorch.image(inp2d)

In [ ]:
?nn.SpatialConvolution

In [ ]:
-- now we build a simple spatial convolution layer.
-- Question: what does it do?
conv2 = nn.SpatialConvolution(1, 1, 3, 3)
print(conv2)
conv2.weight[{1,1,{1,3},{1,3}}] = -1
conv2.weight[{1,1,2,2}] = 8
conv2.bias[1] = 0

In [ ]:
out2d = conv2:forward(inp2d)
itorch.image(out2d)
print(out2d:size())

In [ ]:
-- what about this filter?
conv2.weight[{1,1,{1,3},{1,3}}] = 1./8
itorch.image(conv2:forward(inp2d))

## Intrinsically vector transfer functions
- these layers are typically used in the last (output layer) of the network
- for multiclass classification problems - [SoftMax](https://github.com/torch/nn/blob/master/doc/transfer.md#softmax) layer
- logarithmic verions of above: LogSoftMax are useful in conjunction with the Negative Log Likelihood loss function

In [ ]:
-- softmax converts an output vector into a probability distribution:
smax = nn.SoftMax()

-- try different inputs here, observe that (1) sum is always 1 and (2) entries are always non-negative
x = torch.Tensor{-4.4, 2, 1}
y = smax:forward(x)

-- all elements 
print(y)
print(torch.sum(y))

# Important loss functions [a.k.a. criteria]
- [Mean Square Error](https://github.com/torch/nn/blob/master/doc/criterion.md#nn.MSECriterion)
- [Negative Log Likelihood](https://github.com/torch/nn/blob/master/doc/criterion.md#nn.ClassNLLCriterion) (for use with LogSoftMax layer as it expects logarithms on input)
- [Cross Entropy](https://github.com/torch/nn/blob/master/doc/criterion.md#crossentropycriterion) combines LogSoftMax with NLL

In [ ]:
loss = nn.MSECriterion()
y = torch.Tensor{-1}
t = torch.Tensor{3}
print('MSE loss when output = ' .. y[1] .. ' and target = ' .. t[1] .. ' is ' .. loss:forward(y, t))

In [ ]:
-- Negative log likelihood penalizes wrong predictions 
loss = nn.ClassNLLCriterion()
x = torch.Tensor{1,2,4,0.1}
print('Linear layer output is ' .. utils.vec2str(x))
lsm = nn.LogSoftMax()
y = lsm:forward(x) -- logarithms of the predictions
print('LogSoftMax layer output is ' .. utils.vec2str(y))
t = 3 -- correct class is 3
print('NLL loss assuming target = ' .. t .. ' is ' .. loss:forward(y, t))

In [ ]:
-- how does the loss backpropagate?
-- to increase the loss the output from logsoftmax would have to be decreased in the 3rd element
-- (as a result, this would make the correct class less probable)
dloss_dy = loss:backward(y, t)
print(dloss_dy)

-- we backpropagate through the logsoftmax nonlinearity, observe how the signal propagates into all
-- fields
dloss_dx = lsm:backward(x, dloss_dy)
print(dloss_dx)

In [ ]:
-- cross entropy combines logsoftmax + class negative log likelihood
loss = nn.CrossEntropyCriterion()
-- let's verify that the loss is the same
print(loss:forward(x, t))

In [ ]:
-- and that the gradient is the same
dloss_dx = loss:backward(x, t)
print(dloss_dx)